### Data Wrangling

In [19]:
# all modules
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns
import numpy as np

In [20]:
# global stuff
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 13)
pd.set_option('display.max_columns', 13)

In [21]:
# load dataset
jobs_db = pd.read_csv(r'../data/processed/jobs_db.csv')

In [22]:
api_data=pd.read_json(r'../data/processed/api_data.json')
api_data

,normalized_job_title,parent_uuid,suggestion,uuid
0,data operations director,bb4e5066e69e4228172b4850d2a920f4,Data Operations Director,8fd068c8d9be73abfa678856177b6c40
1,data processing manager,bb4e5066e69e4228172b4850d2a920f4,Data Processing Manager,ac47656fd51c2cd8037057262c910dc4
10,database engineer,596226d5f9ef63de41a852826d483d19,Database Engineer,16b67d7f57698455aefbc1ae1b358b50
100,data conversion operator,78e0562032df97cc82fcc48388cdf34b,Data Conversion Operator,04b0eaf624b2deb45ccd62cf6ec7c7bc
101,data processing control clerk,9b44dbc6767274d8b3a820e133803d00,Data Processing Control Clerk,2616d64becb4f2c478dd42f501bbfbff
102,maintenance data analyst,9b44dbc6767274d8b3a820e133803d00,Maintenance Data Analyst,25fe195cbf915c0824fa6c44e6d0008c
...,...,...,...,...
94,health data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Health Data Analyst,6cf4d336112984368f83181ac5ffe948
95,medical data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Medical Data Analyst,7acaed29536648e45ea2b82d45047fcf
96,data processing services sales representative,96fe8c70b3c2f9ca197722fe88c5d6ca,Data Processing Services Sales Representative,f670246beda8187fbe875401e7bc4ba6


In [23]:
url='https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html=requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
countries = soup.find_all('td')
lista=[]
lista=[i.text for i in countries]
cleaned=[i.strip() for i in lista if len(i)>3]
split=2
country_codes=[cleaned[i:i+split] for i in range(0,len(cleaned),split)]

In [24]:
country_codes=pd.DataFrame(country_codes)
country_codes=country_codes.rename(columns={0:'country',1:'country_code'})
country_codes['country_code']=country_codes['country_code'].str.replace(')','')
country_codes['country_code']=country_codes['country_code'].str.replace('(','')
country_codes

,country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
...,...,...
66,New Zealand,NZ
67,Taiwan,TW
68,Canada,CA


In [25]:
#Rename columnsdf=pd.merge(jobs_df,api_data, left_on='uuid',right_on='uuid')
jobs_db=jobs_db.rename(columns={'uuid':'person_id','dem_education_level':'ed_level','dem_full_time_job':'job_type','question_bbi_2016wave4_basicincome_awareness':'bi_awareness','question_bbi_2016wave4_basicincome_vote':'bi_vote','question_bbi_2016wave4_basicincome_effect':'bi_effect','question_bbi_2016wave4_basicincome_argumentsfor':'bi_argsfor','question_bbi_2016wave4_basicincome_argumentsagainst':'bi_argsagsagainst'})
jobs_db.columns

Index(['person_id', 'country_code', 'rural', 'uuid.1', 'ed_level', 'job_type',
       'normalized_job_code', 'uuid.2', 'age', 'gender', 'dem_has_children',
       'age_group', 'uuid.3', 'bi_awareness', 'bi_vote', 'bi_effect',
       'bi_argsfor', 'bi_argsagsagainst'],
      dtype='object')

In [26]:
#Drop unnecessary columns
drop_cols=[i for i in list(jobs_db.columns) if i.startswith('uuid')]
jobs_db=jobs_db.drop(columns=drop_cols)
jobs_db.columns

Index(['person_id', 'country_code', 'rural', 'ed_level', 'job_type',
       'normalized_job_code', 'age', 'gender', 'dem_has_children', 'age_group',
       'bi_awareness', 'bi_vote', 'bi_effect', 'bi_argsfor',
       'bi_argsagsagainst'],
      dtype='object')

In [27]:
jobs_db=jobs_db.rename(columns={'normalized_job_code':'uuid'})
jobs_db['uuid']

0                                    NaN
1       861a9b9151e11362eb3c77ca914172d0
2                                    NaN
3       049a3f3a2b5f85cb2971ba77ad66e10c
4       f4b2fb1aa40f661488e2782b6d57ad2f
5       27af8700f5577cec835acee2cb90a2ff
                      ...               
9643                                 NaN
9644    847165cfda6b1dc82ae22b967da8af2f
9645    a4d5b8b38f9513825d0d94a981ebe962
9646                                 NaN
9647    775190277a849cba701b306a7b374c0a
9648    8a7a3f1bce0958ae9f090683e5a925b7
Name: uuid, Length: 9649, dtype: object

In [28]:
filtered=jobs_db.uuid.notnull()
jobs_db=jobs_db[filtered]

In [29]:
api_data['uuid']
api_data.size

624

In [30]:
df=pd.merge(jobs_db,api_data, left_on='uuid',right_on='uuid')
df

,person_id,country_code,rural,ed_level,job_type,uuid,...,bi_effect,bi_argsfor,bi_argsagsagainst,normalized_job_title,parent_uuid,suggestion
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
3,a6361230-da58-0133-cd5a-0a81e8b09a82,BE,urban,medium,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,BG,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
5,c878db70-d9a7-0133-5c31-0a81e8b09a82,CZ,Non-Rural,medium,yes,861a9b9151e11362eb3c77ca914172d0,...,‰Û_ work less,It reduces anxiety about financing basic needs...,It might encourage people to stop working | On...,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5696,94332cb0-d9bc-0133-b92b-0a81e8b09a82,IT,city,low,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ gain additional skills,It creates more equality of opportunity | It e...,Only the people who need it most should get so...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk
5697,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,IT,Country,high,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ look for a different job,It reduces anxiety about financing basic needs...,Only the people who need it most should get so...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk
5698,c8c33390-da69-0133-063a-0a81e8b09a82,PL,city,medium,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ look for a different job,It reduces anxiety about financing basic needs,It might encourage people to stop working | It...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk


In [31]:
df.isnull().sum()

person_id                 0
country_code              0
rural                     0
ed_level                256
job_type                  0
uuid                      0
                       ... 
bi_effect                 0
bi_argsfor                0
bi_argsagsagainst         0
normalized_job_title      0
parent_uuid               0
suggestion                0
Length: 18, dtype: int64

In [32]:
df.shape

(5702, 18)

In [33]:
#CLEANING
df.isnull().sum()

person_id                 0
country_code              0
rural                     0
ed_level                256
job_type                  0
uuid                      0
                       ... 
bi_effect                 0
bi_argsfor                0
bi_argsagsagainst         0
normalized_job_title      0
parent_uuid               0
suggestion                0
Length: 18, dtype: int64

In [38]:
df.pivot_table('ed_level', 'suggestion',aggfunc=np.value_counts)

AttributeError: module 'numpy' has no attribute 'value_counts'

In [28]:
df.groupby('suggestion').agg('ed_level').count()

suggestion
Analytical Data Miner                                                           10
Automatic Data Processing Customer Liaison (ADP Customer Liaison)               88
Automatic Data Processing Planner                                               78
Automatic Data Processing Planner (ADP Planner)                                 77
Automatic Data Processing Systems Security Specialist (ADP Systems Security)    85
Business Database Analyst                                                       12
                                                                                ..
SCADA Technician (Supervisory Control and Data Acquisition Technician)          13
SQL Database Administrator                                                      65
Scientific Database Curator                                                      8
Survey Data Technician                                                          11
Voice and Data Technician                                                   

In [21]:
filtro=df['ed_level']=='no'
df[filtro]['suggestion'].unique()

array(['Automatic Data Processing Planner', 'Data Coordinator',
       'Database Developer', 'Data Entry Specialist',
       'Database Architect',
       'Geographic Information Systems Database Administrator (GIS Database Administrator)',
       'Crime Data Specialist', 'Data Communications Software Consultant',
       'Data Security Analyst', 'Database Marketing Analyst',
       'Computer or Data Processing Systems Consultant',
       'Database Development and Administration Project Manager',
       'Naval Tactical Data System--Combat Information Center Watch Officer, General',
       'Data Coder Operator',
       'Automatic Data Processing Customer Liaison (ADP Customer Liaison)',
       'Data Entry Representative', 'Data Processing Auditor',
       'Maintenance Data Analyst',
       'Automatic Data Processing Systems Security Specialist (ADP Systems Security)',
       'Database Engineer', 'Data Processing Systems Project Planner',
       'Database Administration Associate', 'Data W

In [22]:
df.groupby(['ed_level']).size()

ed_level
high      2450
low        777
medium    2111
no         108
dtype: int64

In [21]:
df[filter_c].groupby(['ed_level']).size()

ed_level
high      138
low        77
medium    146
no         15
dtype: int64

In [17]:
df=df.replace({'countryside':'rural', 'city':'Non-rural','urban':'Non-rural','Country':'rural','Rural':'rural','Non-Rural':'Non-rural'})

In [43]:
df['ed_level'].unique()

array(['high', 'medium', 'low', nan, 'no'], dtype=object)